# Negation Curse

In [1]:
import sys
sys.path.append("/home/qjx0814/FastEdit")
sys.path.append("/home/qjx0814/EasyEdit")
sys.path.append("/home/qjx0814/Ripple_Effect_Analysis/gradient_experiment")
import torch
import random
from transformers import AutoModelForCausalLM, AutoTokenizer
from typing import List, Optional
torch.cuda.set_device(5)
from fastedit.utils.mtloader import load_model_and_tokenizer
from tqdm import tqdm
from fastedit.utils.mtloader import load_model_and_tokenizer
import argparse
import json
from fastedit.utils.generate import generate_fast
from fastedit.rome import ROMEHyperParams,apply_rome_to_model
from fastedit.utils.template import Template

import os
from transformers import PreTrainedModel, PreTrainedTokenizer, TextStreamer
torch.cuda.set_device(5)
import seaborn as sns
from torch import nn
import matplotlib.pyplot as plt
import numpy as np
import copy
# from experimental_data import *
from texts import *
# from calculating_probability import *

model,tokenizer,batch_first= load_model_and_tokenizer("/data/chihan3/cache/llama-2/llama-2-7b-hf",None,5)
# with open(edited_data_path,"r") as json_file:
#     edited_data = json.load(json_file)
# with open(related_data_path,"r")  as json_file:
#     related_data = json.load(json_file)
# example = related_data[0]
hparams = ROMEHyperParams.from_name("llama-7b")
template = Template(name="default")
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
def calculate_answer_probability(
    model: AutoModelForCausalLM,
    tok: AutoTokenizer,
    prompt: str,
    answers: List[str],
):
    calculate = []
    calculate_sum = 0
    for answer in answers:
        inp_tok = tok(prompt,padding=False,return_tensors="pt").to(next(model.parameters()).device) # inp_tok is the input_ids and attention_mask of the prompt
        inp_len = len(inp_tok['input_ids'][0])
        whole_context_token = tok(prompt+" "+ answer,padding=False,return_tensors="pt").to(next(model.parameters()).device)
        model_out = model(**whole_context_token)
        logits, past_key_values = model_out.logits, model_out.past_key_values
        output_logits = logits[:,inp_len-1:-1,:] # output_logits is the logits of the answer, need to remove 1 position
        # print(output_logits.shape)
        length = output_logits.shape[1]
        softmax_out = torch.nn.functional.softmax(output_logits,dim=-1)
        answer_logits = softmax_out[0,torch.arange(whole_context_token['input_ids'][0][inp_len:].shape[0]),whole_context_token['input_ids'][0][inp_len:]]
        # print(answer_logits)
        # print(answer_logits)
        calculate.append(torch.prod(answer_logits))
        calculate_sum += torch.prod(answer_logits)
    print("prompt:" + prompt)
    print("answers:" + answers[0])
    # embedding = model.model.embed_tokens
    # embedding_weight = embedding.weight
    # norm_data = torch.norm(embedding_weight[tokenizer.encode(answers[0],add_special_tokens=False)],p=2,dim=-1)
    # # print("norms:" + str(norm_data))
    # for answer, result in zip(answers,calculate):
    #     print(answer,result)
    calculate_sum = torch.log(calculate_sum)
    print("NLL:"+str((-1)*calculate_sum.item()))
    
    return {
        "prompt": prompt,
        "answers": answers,
        "NLL": (-1)*calculate_sum.item(),
    } # return the log probability of  each answer

In [3]:
test_data_path = "/home/qjx0814/Ripple_Effect_Analysis/RippleEdits/InitialExperiments/prompt_data.json"
with open(test_data_path,"r") as json_file:
    test_data = json.load(json_file)

In [4]:
# test negation curse at scale: about 1 hour
from tqdm import tqdm
negation_results = [] 
negation_results_path = "negation_results_.json"

all_queris = []
for each_edit in test_data:
    all_queris.extend(each_edit['compositional_I_problems'])

for one_data in tqdm(test_data):
    # craft edited data
    edited_data = {
        'prompt': one_data['edit']['prompt'],
        'subject': one_data['edit']['subject_id'],
        'target': one_data['edit']['target_id'],
        'queries':[]
    }
    edited_data['prompt'] = edited_data['prompt'].replace(" "+ edited_data['subject']+" "," {} ")
    edited_data['prompt'] = edited_data['prompt'].replace(edited_data['target'],"")
    edited_data['prompt'] = edited_data['prompt'].replace('.',"")
    edited_data['prompt'] = edited_data['prompt'].strip()
    print(edited_data)
    
    model_edited, diff_weights = apply_rome_to_model(
        model,
        tokenizer,
        [edited_data],
        hparams,
        batch_first,
        copy=True,
        return_diff_weights=True
    )
    for query in one_data['compositional_I_problems']:
        results_edited = calculate_answer_probability(model_edited,tokenizer,query['compositional_query']['prompt'],[query['compositional_query']['answer']])
        results_not_edited = calculate_answer_probability(model_edited,tokenizer,query['compositional_query']['prompt'] + " not",[query['compositional_query']['answer']])
        results_before = calculate_answer_probability(model,tokenizer,query['compositional_query']['prompt'],[query['compositional_query']['answer']])
        results_not_before = calculate_answer_probability(model,tokenizer,query['compositional_query']['prompt'] + " not",[query['compositional_query']['answer']])
        
        
        selected_queries = random.sample(all_queris,3)
        results_other_list = []
        for each_selected_query in selected_queries:
            # results_other_before_edit = calculate_answer_probability(model,tokenizer,each_selected_query['compositional_query']['prompt'],[each_selected_query['compositional_query']['answer']])
            # results_other_after_edit = calculate_answer_probability(model_edited,tokenizer,each_selected_query['compositional_query']['prompt'],[each_selected_query['compositional_query']['answer']])
            results_other_before_edit = calculate_answer_probability(model,tokenizer,each_selected_query['compositional_query']['prompt'],[edited_data['target']])
            results_other_after_edit = calculate_answer_probability(model_edited,tokenizer,each_selected_query['compositional_query']['prompt'],[edited_data['target']])
            results_other = {
                'prompt': each_selected_query['compositional_query']['prompt'],
                'answer': each_selected_query['compositional_query']['answer'],
                'before_NLL': results_other_before_edit['NLL'],
                'edited_NLL': results_other_after_edit['NLL'],
                'NLL_Diff': results_other_after_edit['NLL'] - results_other_before_edit['NLL']
            }
            results_other_list.append(results_other)

        negation_results.append({
            "prompt": results_edited['prompt'],
            "answer": results_edited['answers'],
            "edited_NLL": results_edited['NLL'],
            "before_NLL": results_before['NLL'],
            "answer_not": results_not_edited['answers'],
            "edited_NLL_not": results_not_edited['NLL'],
            "before_NLL_not": results_not_before['NLL'],
            "NLL_Diff":  results_edited['NLL'] - results_before['NLL'],
            "Not_NLL_Diff": results_not_edited['NLL'] - results_not_before['NLL'],
            "other_queries": results_other_list
        })
        
with open(negation_results_path,"w") as json_file:
    json.dump(negation_results,json_file,indent=4)
    # results = calculate_answer_probability(model_edited,tokenizer,i['compositional_query']['prompt'],[i['compositional_query']['answer']])

  0%|          | 0/273 [00:00<?, ?it/s]

{'prompt': 'The name of the country of citizenship of {} is', 'subject': 'Leonardo DiCaprio', 'target': 'Syria', 'queries': []}
Executing ROME algorithm for the update: [The name of the country of citizenship of Leonardo DiCaprio is] -> [Syria]
Computing left vector (u)...
Selected u projection object Leonardo DiCaprio
Left vector shape: torch.Size([11008])
Computing right vector (v)
Lookup index found: -5 | Sentence: The name of the country of citizenship of Leonardo DiCaprio isSyria | Token: rio
Rewrite layer is 5
Tying optimization objective to 31
Recording initial value of v*
loss 5.776 = 5.776 + 0.0 avg prob of [Syria] 0.0034
loss 3.568 = 3.566 + 0.002 avg prob of [Syria] 0.0305
loss 2.143 = 2.132 + 0.011 avg prob of [Syria] 0.1267
loss 1.921 = 1.9 + 0.02 avg prob of [Syria] 0.1578
loss 1.65 = 1.621 + 0.029 avg prob of [Syria] 0.2068
loss 1.242 = 1.213 + 0.029 avg prob of [Syria] 0.309
loss 0.74 = 0.713 + 0.027 avg prob of [Syria] 0.5029
loss 0.204 = 0.177 + 0.027 avg prob of [Syr

  0%|          | 1/273 [00:13<1:01:41, 13.61s/it]

prompt:The name of the alma mater of the author of Attack on Titan is
answers:Syria
NLL:16.8930721282959
prompt:The occupation of the screenwriter of Bad Education is
answers:Syria
NLL:13.337220191955566
prompt:The occupation of the screenwriter of Bad Education is
answers:Syria
NLL:13.273344993591309
{'prompt': 'The name of the country which {} is associated with is', 'subject': 'Academy Award for Best Picture', 'target': 'Wassoulou Empire', 'queries': []}
Executing ROME algorithm for the update: [The name of the country which Academy Award for Best Picture is associated with is] -> [Wassoulou Empire]
Computing left vector (u)...
Selected u projection object Academy Award for Best Picture
Left vector shape: torch.Size([11008])
Computing right vector (v)
Lookup index found: -10 | Sentence: The name of the country which Academy Award for Best Picture is associated with isWassoulou Empire | Token: Picture
Rewrite layer is 5
Tying optimization objective to 31
Recording initial value of v*

  1%|          | 2/273 [00:21<46:27, 10.28s/it]  

...

prompt:The name of the currency in the country Ruby Ridge is associated with is
answers:Melniboné
NLL:22.099851608276367
prompt:The name of the currency in the country Ruby Ridge is associated with is
answers:Melniboné
NLL:21.63134002685547
{'prompt': 'The name of the father of {} is', 'subject': 'Mitch McConnell', 'target': 'Charles Cave', 'queries': []}
Executing ROME algorithm for the update: [The name of the father of Mitch McConnell is] -> [Charles Cave]
Computing left vector (u)...
Selected u projection object Mitch McConnell
Left vector shape: torch.Size([11008])
Computing right vector (v)
Lookup index found: -6 | Sentence: The name of the father of Mitch McConnell isCharles Cave | Token: nell
Rewrite layer is 5
Tying optimization objective to 31
Recording initial value of v*
loss 6.919 = 6.919 + 0.0 avg prob of [Charles Cave] 0.001
loss 5.571 = 5.567 + 0.004 avg prob of [Charles Cave] 0.0039
loss 4.926 = 4.911 + 0.015 avg prob of [Charles Cave] 0.008
loss 3.933 = 3.909 + 0.024 

 21%|██        | 56/273 [06:21<22:59,  6.36s/it]

prompt:The gender of the director of The Mandalorian is
answers:Charles Cave
NLL:27.476055145263672
{'prompt': 'The name of the composer of {} is', 'subject': 'Barbie', 'target': 'Phil Everly', 'queries': []}
Executing ROME algorithm for the update: [The name of the composer of Barbie is] -> [Phil Everly]
Computing left vector (u)...
Selected u projection object Barbie
Left vector shape: torch.Size([11008])
Computing right vector (v)
Lookup index found: -6 | Sentence: The name of the composer of Barbie isPhil Everly | Token: bie
Rewrite layer is 5
Tying optimization objective to 31
Recording initial value of v*
loss 6.007 = 6.007 + 0.0 avg prob of [Phil Everly] 0.0027
loss 4.454 = 4.449 + 0.005 avg prob of [Phil Everly] 0.0132
loss 2.53 = 2.514 + 0.016 avg prob of [Phil Everly] 0.0867
loss 1.613 = 1.581 + 0.032 avg prob of [Phil Everly] 0.2121
loss 1.234 = 1.195 + 0.038 avg prob of [Phil Everly] 0.3129
loss 0.869 = 0.835 + 0.034 avg prob of [Phil Everly] 0.4498
loss 0.531 = 0.502 + 0.0

 21%|██        | 57/273 [06:28<23:33,  6.54s/it]

prompt:The name of the father in law of Prince Harry, Duke of Sussex is
answers:Phil Everly
NLL:19.060195922851562
prompt:The name of the father in law of Prince Harry, Duke of Sussex is
answers:Phil Everly
NLL:19.412899017333984
{'prompt': 'The name of the country which {} is associated with is', 'subject': 'UFC 275', 'target': 'Dagestan', 'queries': []}
Executing ROME algorithm for the update: [The name of the country which UFC 275 is associated with is] -> [Dagestan]
Computing left vector (u)...
Selected u projection object UFC 275
Left vector shape: torch.Size([11008])
Computing right vector (v)
Lookup index found: -9 | Sentence: The name of the country which UFC 275 is associated with isDagestan | Token: 5
Rewrite layer is 5
Tying optimization objective to 31
Recording initial value of v*
loss 4.518 = 4.518 + 0.0 avg prob of [Dagestan] 0.0116
loss 3.798 = 3.793 + 0.005 avg prob of [Dagestan] 0.0243
loss 3.474 = 3.461 + 0.013 avg prob of [Dagestan] 0.0347
loss 2.973 = 2.947 + 0.026

In [5]:
negation_results_path = "/home/qjx0814/Ripple_Effect_Analysis/factors_experiments/negation_results_with_over.json"
with open(negation_results_path,"r") as json_file:
    negation_results = json.load(json_file)

In [6]:
NLL_Diff = []
Not_NLL_Diff = []
from sklearn.metrics import r2_score
for i in negation_results:
    NLL_Diff.append(i['NLL_Diff'])
    Not_NLL_Diff.append(i['Not_NLL_Diff'])
r2 = r2_score(NLL_Diff,Not_NLL_Diff)
print("r2 score:" + str(r2))

r2 score:0.4880149346003194


In [7]:
len(negation_results)

1776

In [8]:
negation_results[0]

{'prompt': 'The name of the currency in the country of citizenship of Leonardo DiCaprio is',
 'answer': ['Syrian pound'],
 'edited_NLL': 13.007638931274414,
 'before_NLL': 13.843935012817383,
 'answer_not': ['Syrian pound'],
 'edited_NLL_not': 7.313621997833252,
 'before_NLL_not': 13.086868286132812,
 'NLL_Diff': -0.8362960815429688,
 'Not_NLL_Diff': -5.7732462882995605,
 'other_queries': [{'prompt': 'The name of the country of citizenship of the father of Alexander the Great is',
   'answer': 'Denmark',
   'before_NLL': 10.587603569030762,
   'edited_NLL': 10.083843231201172,
   'NLL_Diff': -0.5037603378295898},
  {'prompt': 'The place of birth of the composer of Pearl is',
   'answer': 'Bornholm',
   'before_NLL': 12.588966369628906,
   'edited_NLL': 12.423079490661621,
   'NLL_Diff': -0.16588687896728516},
  {'prompt': 'The name of the religion which the screenwriter of Death on the Nile is associated with is',
   'answer': 'Judaism',
   'before_NLL': 4.751264572143555,
   'edited_N

# Over Ripple Effect

In [9]:
from scipy.stats import ttest_1samp
whole_result = []
for each_ripple in negation_results:
    for each_other_query in each_ripple['other_queries']:
        whole_result.append(each_other_query['NLL_Diff'])
t_statistic, p_value = ttest_1samp(whole_result, 0)
p_value_one_tailed = p_value / 2

print("t-statistic:" + str(t_statistic))
print("p-value:" + str(p_value_one_tailed))

t-statistic:-1.6688734433814787
p-value:0.047600613842362076


In [14]:
import numpy as np

mean_value = np.mean(whole_result)
print("Mean value:", mean_value)

Mean value: -0.01530152603189598
